<a href="https://colab.research.google.com/github/MatiasSingermann/IBODS-AI/blob/main/IA_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importacion e instalacion de librerias

In [1]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 13.1 MB/s 
     |████████████████████████████████| 178 kB 69.1 MB/s 
     |████████████████████████████████| 1.1 MB 64.2 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 138 kB 92.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.15-py3-none-any.whl size=24936 sha256=87cf14dce4ffb5001a96ecf996e522891f661d77c9aa31c16320ad55574308e0
  Stored in directory: /root/.cache/pip/wheels/13/e0/a3/fe183fffc3972cb9264e800763d4bbfff80d5309137c410217
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=62eeeff6836906c421147a14abae93743a7c2d1f7b97c9d688fd364bcba3099c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37

In [2]:
!pip install tflite-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 13.6 MB/s 
     |████████████████████████████████| 238 kB 71.3 MB/s 
     |████████████████████████████████| 127 kB 78.2 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 10.9 MB 26.7 MB/s 
     |████████████████████████████████| 3.4 MB 55.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 840 kB 51.5 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 128 kB 81.6 MB/s 
     |████████████████████████████████| 1.3 MB 43.1 MB/s 
     |████████████████████████████████| 60.2 MB 1.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 352 kB 73.1 MB/s 
     |████████████████████████████████| 99 kB 11.0 MB/s 
     |█████████████████████████████

Importo `Tensorflow` y varios de sus **modulos**.

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tflite_model_maker import object_detector
import tensorflow_hub as hub
from tflite_model_maker.config import QuantizationConfig
from tflite_support.task import vision

Importo varias `librerias` que me permiten trabajar con **tensores**, y **operaciones matematicas** necesrias. 

In [4]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms

Importo `librerias` para **administrar archivos** y seleccionar de manera **random** alguno de ellos

In [5]:
import os
import random

Importo `librerias` que me permiten trabajar con las **imagenes** del dataset.

In [7]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.image import imread
from itertools import product
from skimage import draw, transform

Importo `Roboflow` para acceder al dataset

In [8]:
from roboflow import Roboflow

## Importacion del **Dataset** y sus **labels**

Me conecto a mi cuenta de Drive para agarrar ciertos archivos

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Importo el dataset desde Roboflow

In [9]:
rf = Roboflow(api_key="41l2d60Eq7cWhUOVgPxd")
project = rf.workspace("ibods").project("ibods_lbl")
dataset = project.version(1).download("voc")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to IBODS_LBL-1 in voc:: 100%|██████████| 7031/7031 [00:01<00:00, 5271.84it/s]


Asigno la **ruta del archivo** de las anotaciones a una variable

In [11]:
path_lbl = '/content/gdrive/MyDrive/Proyecto 4to/I.A/Anotaciones/labels.txt'

Abro el **archivo de anotaciones** y lo *recorro linea por linea*, *eliminando los espacios* y metiendo los `strings` en una `lista`

In [12]:
with open(path_lbl) as f:
   labels = [line.strip() for line in f.readlines()]
labels

['cordon',
 'autos',
 'personas',
 'cruces',
 'pozos',
 'parar',
 'cruzar',
 'bicicleta',
 'moto',
 'escalones']

Usando el **arhcivo de labels**, le asigno un *valor numerico* para que la `IA` pueda interpretar las anotaciones

In [13]:
assigned_labels = { i : labels[i] for i in range(0, len(labels) ) }
assigned_labels

{0: 'cordon',
 1: 'autos',
 2: 'personas',
 3: 'cruces',
 4: 'pozos',
 5: 'parar',
 6: 'cruzar',
 7: 'bicicleta',
 8: 'moto',
 9: 'escalones'}

## Funciones Utiles y configuracion

In [14]:
CLASSES = labels

In [15]:
COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3), dtype=np.uint8)

In [22]:
def preprocess_image(image_path, input_size):
  ''' Preprocess the input image to feed to the TFLite model
  '''
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image

In [23]:
def detect_objects(interpreter, image, threshold):
  ''' Returns a list of detection results, each a dictionary of object info.
  '''

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results

In [24]:
def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  ''' Run object detection on the input image and draw the detection results
  '''
  
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(CLASSES[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

## Cargar Dataset

### Cargo los datos de entrenamiento

In [16]:
TRAIN_PATH = 'IBODS_LBL-1/train'

In [17]:
train_data = object_detector.DataLoader.from_pascal_voc(
    TRAIN_PATH,
    TRAIN_PATH,
    labels
)

### Cargo los datos de validacion

In [18]:
VALID_PATH = 'IBODS_LBL-1/valid'

In [19]:
valid_data = object_detector.DataLoader.from_pascal_voc(
    VALID_PATH,
    VALID_PATH,
    labels
)

### Cargo los datos de testeo

In [20]:
TEST_PATH = 'IBODS_LBL-1/test'

In [21]:
test_data = object_detector.DataLoader.from_pascal_voc(
    TEST_PATH,
    TEST_PATH,
    labels
)

## Entrenamiento de la IA

In [ ]:
model = object_detector.create(
    train_data,
    model_spec= 'efficientdet_lite1',
    batch_size= 8,
    train_whole_model=True,
    epochs=100,
    validation_data=valid_data
)

## Testeo de la IA y su performance

In [ ]:
model.evaluate(test_data)

## Salvo el modelo

In [25]:
os.mkdir('logs')

In [ ]:
model.export(export_dir='logs')

In [ ]:
model.evaluate_tflite('logs/model.tflite', test_data)

## Quantization

In [27]:
config = QuantizationConfig.for_float16()

In [38]:
converter = tf.lite.TFLiteConverter.from_saved_model('logs/model1.tflite')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

OSError: ignored

## Deteccion

### Deteccion **simple** fotos

In [ ]:
detector = vision.ObjectDetector.create_from_file('logs/model1.tflite')

In [ ]:
tensor_image = vision.TensorImage.create_from_file('IBODS_LBL-1/test/Audi_A7_2012_59_18_310_30_6_75_55_195_18_AWD_4_4_4dr_aqB_jpg.rf.2cdf87614d0d1c937d1e97bbdd338b2c.jpg')

In [ ]:
detector.detect(tensor_image)

### Deteccion **compleja** fotos

In [ ]:
cwd = os.getcwd()
os.mkdir(f'{cwd}/result')

In [ ]:
cwd = os.getcwd()

DETECTION_THRESHOLD = 0.3

# Change the test file path to your test image
INPUT_IMAGE_PATH = 'IBODS_LBL-1/test/Audi_A7_2012_59_18_310_30_6_75_55_195_18_AWD_4_4_4dr_aqB_jpg.rf.2cdf87614d0d1c937d1e97bbdd338b2c.jpg'

im = Image.open(INPUT_IMAGE_PATH)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(f'{cwd}/result/input.png','PNG')

# Load the TFLite model
model_path = 'logs/model1.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_PATH,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
img = Image.fromarray(detection_result_image)
img.save(f'{cwd}/result/ouput.png')

### Deteccion en video

In [ ]:
detector = vision.ObjectDetector.create_from_file('logs/model1.tflite')

import cv2

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se puede abrir la camara")
    exit()

while True:
  ret,frame = cap.read() # Voy agarrando los frames

  if ret == True: # Si leyo correctamente los frames, signfica que el retun (ret) es true, por lo que ejecuto el codigo en ese caso
    cv2.cvtColor(frame, cv2.COLOR_BGRRGB) # Convierto todos los frames al espectro RGB
    tensor_image = vision.TensorImage.create_from_array(frame) # Convierto todos los frames a tensores
    detector.detect(tensor_image)
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  if not ret:
        print("No puedo recibir imagenes (finalizaste la trasmision?). Saliendo ...")
        break
cap.release()
cv2.destroyAllWindows()